## Grouping data with Pandas

The groupby clause is an operation on DataFrames. A Series is a 1D object, so performing a groupby operation on it is not very useful. However, it can be used to obtain distinct rows of the Series. The result of a groupby operation is not a DataFrame but dict of DataFrame objects. Let us start with a dataset involving the world's most popular sport—soccer.

### Data Acquistion

This dataset, obtained from Wikipedia, contains data for the finals of the European club championship since its inception in 1955. For reference, you can go to this [link] (http://en.wikipedia.org/wiki/UEFA_Champions_League) 

First we will create a new folder to store the dataset in our root setting a condition if the path does not exist, we'll create it, by using the _os_ package. Then we will use _urllib_ package to download the csv file from *base_url*. Then we'll store it in the folder we previously created, before that we'll make sure that the file does not exist.

Once get get our dataset, we;ll convert it into a DataFrame by using *read_csv* command

In [24]:
# Importing urlib
import urllib
import pandas as pd
import os

# Creating the data folder
if not os.path.exists('./data'):
    os.makedirs('./data')

# Obtaining the dataset using the url that hosts it
base_url = 'https://github.com/fvgm-spec/Pandas/blob/master/uefa.csv'
if not os.path.exists('./data/uefa.csv'):     # avoid downloading if the file exists
    response = urllib.request.urlretrieve(base_url, './data/uefa.csv')

In the output of the previous code we get a DataFrame wth 64 rows that shows the data for all Champions League finals from the beginning of the competition won by Real Madrid until the last edition won by Liverpool.

In [54]:
uefaDF=pd.read_excel('C:/Data/csv/uefa_champions_winners.xlsx')
uefaDF

,Season,Nation,Winners,Score,Runners-up,Runner-UpNation,Venue,Attendance
0,1955–56,Spain,Real Madrid,4–3,Stade de Reims,France,"Parc des Princes,Paris",38239
1,1956–57,Spain,Real Madrid,2–0,Fiorentina,Italy,"Santiago Bernabéu Stadium, Madrid",124000
2,1957–58,Spain,Real Madrid,3–2,Milan,Italy,"Heysel Stadium,Brussels",67000
3,1958–59,Spain,Real Madrid,2–0,Stade de Reims,France,"Neckarstadion,Stuttgart",72000
4,1959–60,Spain,Real Madrid,7–3,Eintracht Frankfurt,Germany,"Hampden Park,Glasgow",127621
...,...,...,...,...,...,...,...,...
59,2014–15,Spain,Barcelona,3–1,Juventus,Italy,"Olympic Stadium,Berlin",70442
60,2015–16,Spain,Real Madrid,1–1*[J],Atletico Madrid,Spain,Giuseppe Meazza,71942
61,2016–17,Spain,Real Madrid,4–1,Juventus,Italy,Cardiff Stadium,65842
62,2017–18,Spain,Real Madrid,4–1,Liverpool,England,Olimpiyskiy NCS Stadium,61561


### Start Grouping Data

Thus, the output shows the season, the nations to which the winning and runner-up clubs belong, the score, the venue, and the attendance figures. Suppose we wanted to rank the nations by the number of European club championships they had won. We can do this by using groupby. First, we apply groupby to the DataFrame and see what is the type of the result:

In [127]:
nationsGrp =uefaDF.groupby(['Nation'])
nationsGrp

Thus, we see that **nationsGrp** is of the _pandas.core.groupby.DataFrameGroupBy_ type. The column on which we use groupby is referred to as the key. We can see what the groups look like by using the groups attribute on the resulting _DataFrameGroupBy_ object:

In [108]:
nationsGrp.groups

{'England': Int64Index([12, 21, 22, 23, 24, 25, 26, 28, 43, 49, 52, 56, 63], dtype='int64'),
 'France': Int64Index([37], dtype='int64'),
 'Germany': Int64Index([18, 19, 20, 27, 41, 45, 57], dtype='int64'),
 'Italy': Int64Index([7, 8, 9, 13, 29, 33, 34, 38, 40, 47, 51, 54], dtype='int64'),
 'Netherlands': Int64Index([14, 15, 16, 17, 32, 39], dtype='int64'),
 'Portugal': Int64Index([5, 6, 31, 48], dtype='int64'),
 'Romania': Int64Index([30], dtype='int64'),
 'Scotland': Int64Index([11], dtype='int64'),
 'Spain': Int64Index([0, 1, 2, 3, 4, 10, 36, 42, 44, 46, 50, 53, 55, 58, 59, 60, 61, 62], dtype='int64'),
 'Yugoslavia': Int64Index([35], dtype='int64')}

This is basically a dictionary that just shows the unique groups and the axis labels corresponding to each group—in this case the row number. We can get for example whole information of the 62th index of the DataFrame, that corresponds to 2017-18 season final, which represented the 13th "Orejona" for Real Madrid disputed against Liverpool.

The number of groups is obtained by using the *len()* function in the cell below:

In [140]:
Spain = uefaDF.iloc[62]
Spain

Season                             2017–18
Nation                               Spain
Winners                        Real Madrid
Score                                  4–1
Runners-up                       Liverpool
Runner-UpNation                    England
Venue              Olimpiyskiy NCS Stadium
Attendance                           61561
Name: 62, dtype: object

In [142]:
len(nationsGrp.groups)

10

Here the data we grouped previously determined by `DataFrameGroupBy object`, identified with the variable name _nationsGrp_, we'll use it to display some tables, but first we need to convert it to DataFrame, so we can create a new mesure and sort it ascending.

In the table we note that the Nation with more wins in Champions is Spain, mostly due to the 13 a 5 Trophys from Real Madrid and Barcelona.

In [143]:
nationWins=nationsGrp.size().to_frame('Champion')
NationsWinners=nationWins.sort_values(by='Champion', ascending=False)
NationsWinners

,Champion
Nation,
Spain,18
England,13
Italy,12
Germany,7
Netherlands,6
Portugal,4
France,1
Romania,1
Scotland,1


The _size()_ function returns a Series with the group names as the index and the size of each group. The _size()_ function is also an aggregation function.

To do a further breakup of wins by country and club, we apply a **multicolumn groupby function** and then size() and sort():

In [165]:
winners = uefaDF.groupby(['Nation','Winners']).size().to_frame('Champion')
winnersUEFA = winners.sort_values(by='Champion', ascending=False)
winnersUEFA

,,Champion
Nation,Winners,
Spain,Real Madrid,13
Italy,Milan,7
England,Liverpool,6
Germany,Bayern Munich,5
Spain,Barcelona,5
Netherlands,Ajax,4
England,Manchester United,3
Italy,Internazionale,3
Portugal,Benfica,2


A **multicolumn groupby** specifies more than one column to be used as the key by specifying the key columns as a list. Thus, we can see that the most successful club in this competition has been Real Madrid of Spain.

Now we will examine a richer dataset that will enable us to illustrate many more features of groupby. This dataset is also soccer related and provides statistics for the top four European soccer leagues in the 2012-2013 season:
* English Premier League or EPL
* Spanish Primera Division or La Liga
* Italian First Division or Serie A
* German Premier League or Bundesliga

The source of this information is at http://soccerstats.com.
Let us now read the goal stats data into a DataFrame as usual. In this case, we create a row index on the DataFrame using the _Month_ column.

In [158]:
goalStatsDF=pd.read_csv('C:/Data/csv/goals_stats_euro_leagues.csv')
goalStatsDF=goalStatsDF.set_index('Month')

At first let's take a snapshot of the head and tail ends of our dataset:

In [157]:
goalStatsDF.head(3)

,Stat,EPL,La Liga,Serie A,Bundesliga
Month,,,,,
08/01/2012,MatchesPlayed,20.0,20,10.0,10.0
09/01/2012,MatchesPlayed,38.0,39,50.0,44.0
10/01/2012,MatchesPlayed,31.0,31,39.0,27.0


In [152]:
goalStatsDF.tail(3)

,Stat,EPL,La Liga,Serie A,Bundesliga
Month,,,,,
04/01/2013,GoalsScored,105.0,127,102.0,104.0
05/01/2013,GoalsScored,96.0,109,102.0,92.0
06/01/2013,GoalsScored,NaN,80,NaN,NaN


There are two measures in this data frame _MatchesPlayed_ and _GoalsScored_ and the data is ordered first by Stat and then by Month. Note that the last row in the tail() output has the NaN values for all the columns except La Liga but we'll discuss this in more detail later. We can use groupby to display the stats, but this will be done by grouped year instead. Here is how this is done:

In [159]:
goalStatsGroupedByYear = goalStatsDF.groupby(lambda Month: Month.split('/')[2])
goalStatsGroupedByYear

We can then iterate over the resulting _DataFrameGroupBy object_ and display the groups. In the following command, we see the two sets of statistics grouped by year. Note the use
of the lambda function to obtain the year group from the first day of the month.

In [155]:
for name, group in goalStatsGroupedByYear:
    print(name)
    print(group)

2012
                     Stat    EPL  La Liga  Serie A  Bundesliga
Month                                                         
08/01/2012  MatchesPlayed   20.0       20     10.0        10.0
09/01/2012  MatchesPlayed   38.0       39     50.0        44.0
10/01/2012  MatchesPlayed   31.0       31     39.0        27.0
11/01/2012  MatchesPlayed   50.0       41     42.0        46.0
12/01/2012  MatchesPlayed   59.0       39     39.0        26.0
08/01/2012    GoalsScored   57.0       60     21.0        23.0
09/01/2012    GoalsScored  111.0      112    133.0       135.0
10/01/2012    GoalsScored   95.0       88     97.0        77.0
11/01/2012    GoalsScored  121.0      116    120.0       137.0
12/01/2012    GoalsScored  183.0      109    125.0        72.0
2013
                     Stat    EPL  La Liga  Serie A  Bundesliga
Month                                                         
01/01/2013  MatchesPlayed   42.0       40     40.0        18.0
02/01/2013  MatchesPlayed   30.0       40    

If we wished to group by individual month instead, we would need to apply groupby with a level argument, as follows:

In [160]:
goalStatsGroupedByMonth = goalStatsDF.groupby(level=0)

In [161]:
for name, group in goalStatsGroupedByMonth:
    print(name)
    print(group)
    print("\n")

01/01/2013
                     Stat    EPL  La Liga  Serie A  Bundesliga
Month                                                         
01/01/2013  MatchesPlayed   42.0       40     40.0        18.0
01/01/2013    GoalsScored  117.0      121    104.0        51.0


02/01/2013
                     Stat   EPL  La Liga  Serie A  Bundesliga
Month                                                        
02/01/2013  MatchesPlayed  30.0       40     40.0        36.0
02/01/2013    GoalsScored  87.0      110    100.0       101.0


03/01/2013
                     Stat   EPL  La Liga  Serie A  Bundesliga
Month                                                        
03/01/2013  MatchesPlayed  35.0       38     39.0        36.0
03/01/2013    GoalsScored  91.0      101     99.0       106.0


04/01/2013
                     Stat    EPL  La Liga  Serie A  Bundesliga
Month                                                         
04/01/2013  MatchesPlayed   42.0       42     41.0        36.0
04/01/2013   

Note that since in the preceding commands we're grouping on an index, we need to specify the level argument as opposed to just using a column name. When we group by multiple keys, the resulting group name is a tuple, as shown in the upcoming commands. First, we reset the index to obtain the original DataFrame and define a MultiIndex in order to be able to group by multiple keys.

In [162]:
goalStatsDF=goalStatsDF.reset_index()
goalStatsDF=goalStatsDF.set_index(['Month','Stat'])

In [163]:
monthStatGroup=goalStatsDF.groupby(level=['Month','Stat'])

In [164]:
for name, group in monthStatGroup:
    print(name)
    print(group)

('01/01/2013', 'GoalsScored')
                          EPL  La Liga  Serie A  Bundesliga
Month      Stat                                            
01/01/2013 GoalsScored  117.0      121    104.0        51.0
('01/01/2013', 'MatchesPlayed')
                           EPL  La Liga  Serie A  Bundesliga
Month      Stat                                             
01/01/2013 MatchesPlayed  42.0       40     40.0        18.0
('02/01/2013', 'GoalsScored')
                         EPL  La Liga  Serie A  Bundesliga
Month      Stat                                           
02/01/2013 GoalsScored  87.0      110    100.0       101.0
('02/01/2013', 'MatchesPlayed')
                           EPL  La Liga  Serie A  Bundesliga
Month      Stat                                             
02/01/2013 MatchesPlayed  30.0       40     40.0        36.0
('03/01/2013', 'GoalsScored')
                         EPL  La Liga  Serie A  Bundesliga
Month      Stat                                           
03/01